3. Faça uma tabela com média e desvio-padrão das variáveis. Faça outra tabela com a correlação entre S e IQ.

In [33]:
import pandas as pd

df = pd.read_excel('grilic.xls')


In [34]:
parametros = df.agg(['mean', 'std'], numeric_only=True)

correlacao_S_IQ = df['S'].corr(df['IQ'])


4. Apresente uma análise da escolha dos instrumentos para resolver os problemas de viés de variável omitida e erro-nas-variáveis. Inclua nesta análise a regressão de primeiro estágio que achar pertinente.

Instrumentos usados poderiam ser KWW, MED

Os instrumentos devem ser correlacionados com as variáveis endógenas (IQ e S) e devem ser exógenos

In [35]:
linhas_interesse = ['S', 'IQ']
colunas_interesse = ['MED', 'KWW', 'MRT', 'AGE']

# Calcular a matriz de correlação completa para todas as colunas relevantes
# Primeiro, criamos uma lista com todas as colunas que participarão
colunas_para_corr_completa = list(set(linhas_interesse + colunas_interesse)) # set() para evitar duplicatas
matriz_corr_completa = df[colunas_para_corr_completa].corr(numeric_only=True)

# Selecionar apenas as correlações de interesse
matriz_corr_especifica = matriz_corr_completa.loc[linhas_interesse, colunas_interesse]

correlacao_iv = pd.DataFrame(matriz_corr_especifica).T

In [55]:
import statsmodels.api as sm
from patsy import dmatrices

year_dummies = pd.get_dummies(df['YEAR'], prefix='YEAR', drop_first=True, dtype=int)
df_dummies = df.join(year_dummies)

y, X = dmatrices('IQ ~ RNS + SMSA + MED + KWW + MRT + AGE + YEAR_67 + YEAR_68 + YEAR_69 + YEAR_70 + YEAR_71 + YEAR_73', data=df_dummies, return_type='dataframe')

mod = sm.OLS(Y, X)
res = mod.fit()
print(res.summary())


                            OLS Regression Results                            
Dep. Variable:                     IQ   R-squared:                       0.251
Model:                            OLS   Adj. R-squared:                  0.239
Method:                 Least Squares   F-statistic:                     20.82
Date:                Sat, 14 Jun 2025   Prob (F-statistic):           1.06e-39
Time:                        18:30:09   Log-Likelihood:                -2944.9
No. Observations:                 758   AIC:                             5916.
Df Residuals:                     745   BIC:                             5976.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     76.0966      4.187     18.176      0.0

5. A taxa salarial observada é diferente entre indivíduos (datas diferentes para o primeiro ano de observação), isto implica na necessidade de formar variáveis dummies para anos. Assim dummies de anos devem ser incluídas para controlar o efeito do ano. Estime o modelo de salário (1) por 2SLS. Faça uma análise de validade dos instrumentos utilizando a estatística de Sargan. O conjunto de instrumentos deve consistir nos regressores pré-determinados e nas demais variáveis pré-determinadas (MED, KWW, MRT, AGE). Além de apresentar as estimativas do modelo por OLS e 2SLS, explique a diferença de magnitude entre elas.


In [56]:
df_dummies['AGE^2'] = df_dummies['AGE'] ** 2
 
y, X = dmatrices('IQ ~ S + AGE + AGE^2 + KWW + MED + EXPR + TENURE + RNS + SMSA + YEAR_67 + YEAR_68 + YEAR_69 + YEAR_70 + YEAR_71 + YEAR_73', data=df_dummies, return_type='dataframe')

mod = sm.OLS(Y, X)
res = mod.fit()
print(res.summary())


                            OLS Regression Results                            
Dep. Variable:                     IQ   R-squared:                       0.336
Model:                            OLS   Adj. R-squared:                  0.323
Method:                 Least Squares   F-statistic:                     25.03
Date:                Sat, 14 Jun 2025   Prob (F-statistic):           2.66e-56
Time:                        18:30:15   Log-Likelihood:                -2899.4
No. Observations:                 758   AIC:                             5831.
Df Residuals:                     742   BIC:                             5905.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     67.7354      4.008     16.899      0.0